In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/AIPI_590/Final Project/DRL2/preprocessing

/content/drive/MyDrive/AIPI_590/Final Project/DRL2/preprocessing


In [5]:
import numpy as np
import pandas as pd
import datetime

In [9]:
# Load datassets
purchase_df = pd.read_csv('dataset-train/train-purchases.csv', sep=';')
clicks_df = pd.read_csv('dataset-train/train-clicks.csv', sep=';')
views_df = pd.read_csv('dataset-train/train-item-views.csv', sep=';')
queries_df = pd.read_csv('dataset-train/train-queries.csv', sep=';',low_memory=False)

In [10]:
# Exclude test datapoints
queries_df = queries_df[queries_df['is.test']== False]

In [11]:
# Merge clicks and queries dataframes
clicks_ses_df = clicks_df.merge(queries_df[['queryId','eventdate','sessionId']], left_on = "queryId", right_on = "queryId", how = "left")

In [12]:
# Keep columns 'sessionId','eventdate','itemId' inpurchase dataframe
purchase_df = purchase_df[['sessionId','eventdate','itemId']]

In [13]:
# Add column "is_buy"
purchase_df['is_buy'] = 1

In [14]:
# Keep columns 'sessionId','eventdate','itemId' inpurchase dataframe
clicks_ses_df = clicks_ses_df[['sessionId','eventdate','itemId']]

In [15]:
# Add column "is_buy"
clicks_ses_df['is_buy'] = 0

In [16]:
# Concat clicks and purchases
data_df = pd.concat([clicks_ses_df, purchase_df])

In [17]:
# Rename columns
data_df.rename(columns={'sessionId': 'session_id_c',
                   'eventdate': 'timestamp',
                   'itemId':'item_id'},
          inplace=True)

In [22]:
# Sort data
data_sort_df = data_df.sort_values(['session_id_c', 'timestamp'], ascending=[True, True])

In [23]:
data_sort_df= data_sort_df.dropna()
data_sort_df = data_sort_df.reset_index(drop=True)

In [24]:
# Create new colummn for 'session_id' and 'item_id'
data_sort_df['session_id'] = np.array(data_sort_df.session_id_c)
data_sort_df['item_id'] = np.array(data_sort_df.item_id)
# Keep columns 'session_id','timestamp','item_id','is_buy'
data_sort_df = data_sort_df[['session_id','timestamp','item_id','is_buy']]
# data_sort_df['is_buy'] = np.array(data_sort_df.is_buy)

In [25]:
# List of counts per session_id
lst_ses = data_sort_df["session_id"].value_counts()
# Exclude all sessions with less than 3 and greather than 50 events
data_sort_df = data_sort_df[(data_sort_df['session_id'].isin(lst_ses[lst_ses>2].index)) & (data_sort_df['session_id'].isin(lst_ses[lst_ses<51].index))]

In [26]:
# List of all unique item_ids
unq_lst = np.array(data_sort_df.item_id.unique())
# Indexes of unique items in data_sort_df
idx = [np.where(unq_lst==i)[0][0] for i in data_sort_df['item_id']]
# Add new column with indexes
data_sort_df['idx'] = idx
# Rename columns
data_sort_df.rename(columns={'item_id': 'item_id_origin',
                   'idx': 'item_id'},
          inplace=True)

In [27]:
# Reset index
data_sort_df = data_sort_df.reset_index(drop=True)
# Keep columns 'session_id', 'timestamp', 'item_id', 'is_buy'
data = data_sort_df[['session_id', 'timestamp', 'item_id', 'is_buy']]

In [28]:
# Create function to save dataframe 
import os
def to_pickled_df(data_directory, **kwargs):
    for name, df in kwargs.items():
        df.to_pickle(os.path.join(data_directory, name + '.df'))

In [ ]:
to_pickled_df('dataset-train', sampled_sessions=data)